In [1]:
import yaml

data = {
    'path': 'Pothole_Datasets',
    'train': 'train/images',
    'val': 'train/images',
    'test': 'test/images',
    'nc': 1,
    'names': ['pothole']
}

with open("pothole.yaml", "w") as f:
    yaml.dump(data, f)

print("pothole.yaml created")


pothole.yaml created


In [2]:
import torch
import os

# 1. Force PyTorch to use the old loading behavior (security bypass)
os.environ["TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD"] = "1"

# 2. Add the missing YOLO components to the safe list
from ultralytics.nn.tasks import DetectionModel
import ultralytics.nn.modules as modules

torch.serialization.add_safe_globals([
    DetectionModel, 
    modules.Conv, 
    modules.block.C2f, 
    modules.block.DFL, 
    modules.head.Detect,
    modules.block.Bottleneck,
    modules.block.SPPF,
    torch.nn.modules.container.Sequential,
    torch.nn.modules.container.ModuleList,
    torch.nn.modules.conv.Conv2d,
    torch.nn.modules.batchnorm.BatchNorm2d,
    torch.nn.modules.activation.SiLU
])

from ultralytics import YOLO

# 3. CRITICAL: Delete the old file so it downloads a fresh one
if os.path.exists('yolov8n.pt'):
    os.remove('yolov8n.pt')

# 4. Try loading again
model = YOLO('yolov8n.pt')
print("Model loaded successfully!")

100%|██████████| 6.23M/6.23M [00:05<00:00, 1.20MB/s]
c:\Users\bilar\anaconda3\Lib\site-packages\ultralytics\nn\tasks.py:567: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  return torch.load(file, map_location='cpu'), file  # load


Model loaded successfully!


In [6]:
from ultralytics import YOLO

# 1. Load the model (use the 'Safe Globals' block from our previous chat first)
model = YOLO('yolov8n.pt') 

# 2. Start training explicitly on CPU
# We reduce imgsz and batch to keep the CPU from overheating
results = model.train(
    data='pothole.yaml',
    epochs=30,           # Start with 30 to see progress
    imgsz=320,           # Smaller image size is MUCH faster on CPU
    batch=4,             # Small batch size for CPU stability
    device='cpu',        # FORCED CPU MODE
    name='pothole_pi_cpu_train'
)

Ultralytics 8.4.12  Python-3.12.1 torch-2.10.0+cu126 CPU (13th Gen Intel Core i5-13450HX)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=pothole.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=pothole_pi_cpu_train11, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspe

KeyboardInterrupt: 

In [5]:
from ultralytics import YOLO
import os

# 1. Load your specifically trained model
# This uses the "brain" created in your 8th training run
model = YOLO('runs/detect/pothole_pi_cpu_train8/weights/best.pt')

# 2. Use your absolute path with an 'r' at the start to avoid Windows errors
image_path = r"C:\python\Ai_SOC\New folder\pothole_sample.jpg"

# 3. Check if the file exists before running
if os.path.exists(image_path):
    # Run prediction (imgsz=320 is safer for CPU on some systems)
    results = model.predict(source=image_path, conf=0.25, save=True, imgsz=320)
    
    print(f"--- SUCCESS ---")
    print(f"Prediction complete. Result saved to: {results[0].save_dir}")
else:
    print(f"❌ Error: The file was not found at {image_path}. Check the folder name carefully!")

AssertionError: Invalid device id

In [7]:
from ultralytics import YOLO

# 1. Load the official base model (it will download if you don't have it)
model = YOLO('yolov8n.pt')

# 2. Run prediction on a built-in URL image
# This avoids any "file path" issues
results = model.predict(
    source='https://ultralytics.com/images/bus.jpg', 
    save=True, 
    device='cpu'
)

# 3. Print the location to confirm it worked
print(f"--- TEST COMPLETE ---")
print(f"If you see this, the model is working!")
print(f"Result saved in: {results[0].save_dir}")


image 1/1 c:\python\Ai_SOC\New folder\bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 140.1ms
Speed: 7.4ms preprocess, 140.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)
Results saved to C:\python\Ai_SOC\New folder\runs\detect\predict12
--- TEST COMPLETE ---
If you see this, the model is working!
Result saved in: C:\python\Ai_SOC\New folder\runs\detect\predict12


In [9]:
from ultralytics import YOLO
import os

# 1. Load your CUSTOM trained pothole model (The "Brain" from Run 8)
model = YOLO('runs/detect/pothole_pi_cpu_train8/weights/best.pt')

# 2. Path to your downloaded video 
# Replace this with your actual video filename (e.g., 'pothole_video.mp4')
video_path = 'pothole_video.mp4' 

if os.path.exists(video_path):
    print(f"🎬 Starting video analysis on: {video_path}")
    
    # 3. Run prediction on the video
    # imgsz=320 makes it faster on your laptop's CPU
    # conf=0.3 filters out weak detections
    results = model.predict(
        source=video_path, 
        save=True, 
        device='cpu', 
        imgsz=320, 
        conf=0.3
    )
    
    print(f"--- VIDEO COMPLETE ---")
    print(f"The detected video is saved in: {results[0].save_dir}")
else:
    print(f"❌ Error: Could not find the file '{video_path}'.")
    print("Make sure the video is in the same folder as your notebook.")

🎬 Starting video analysis on: pothole_video.mp4

WARNING 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/585) c:\python\Ai_SOC\New folder\pothole_video.mp4: 192x320 3 potholes, 48.5ms
video 1/1 (frame 2/585) c:\python\Ai_SOC\New folder\pothole_video.mp4: 192x320 3 potholes, 26.2ms
video 1/1 (frame 3/585) c:\python\Ai_SOC\New folder\pothole_video.mp4: 192x320 3 potholes, 30.8ms
video 1/1 (frame 4/585) c:\python\Ai_SOC\New folder\pothole_video.mp4: 192x320 2 potholes, 26.3ms
video 1/1 (frame 5/58

In [10]:
from ultralytics import YOLO
import os

# 1. Load your CUSTOM trained pothole model (The "Brain" from Run 8)
model = YOLO('runs/detect/pothole_pi_cpu_train8/weights/best.pt')

# 2. Path to your downloaded video 
# Replace this with your actual video filename (e.g., 'pothole_video.mp4')
video_path = 'pothole_video2.mp4' 

if os.path.exists(video_path):
    print(f"🎬 Starting video analysis on: {video_path}")
    
    # 3. Run prediction on the video
    # imgsz=320 makes it faster on your laptop's CPU
    # conf=0.3 filters out weak detections
    results = model.predict(
        source=video_path, 
        save=True, 
        device='cpu', 
        imgsz=320, 
        conf=0.3
    )
    
    print(f"--- VIDEO COMPLETE ---")
    print(f"The detected video is saved in: {results[0].save_dir}")
else:
    print(f"❌ Error: Could not find the file '{video_path}'.")
    print("Make sure the video is in the same folder as your notebook.")

🎬 Starting video analysis on: pothole_video2.mp4

WARNING 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/389) c:\python\Ai_SOC\New folder\pothole_video2.mp4: 192x320 (no detections), 30.9ms
video 1/1 (frame 2/389) c:\python\Ai_SOC\New folder\pothole_video2.mp4: 192x320 3 potholes, 23.8ms
video 1/1 (frame 3/389) c:\python\Ai_SOC\New folder\pothole_video2.mp4: 192x320 2 potholes, 26.1ms
video 1/1 (frame 4/389) c:\python\Ai_SOC\New folder\pothole_video2.mp4: 192x320 2 potholes, 28.3ms
video 1/1 (